In [236]:
import sys
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import math
from datetime import datetime, timedelta, time, date

from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
%matplotlib inline

In [223]:
# 使用LSTM进行预测，主要参考了pytorch的examples中的time_series_prediction

In [224]:
# using KNN to predict
train_path = '../dataset/training/volume(table 6)_training.csv'
test_path = '../dataset/testing_phase1/volume(table 6)_test1.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df.time = pd.to_datetime(train_df.time)
test_df.time = pd.to_datetime(test_df.time)

In [225]:
# from 9-19 to 10-17 (except holiday)
NUM_TRAIN_DAYS = 20

# from 10-18 to 10-24
NUM_TSET_DAYS = 7

# define Holiday
NATIONNAL_START = date(2016,10,1)
NATIONNAL_END = date(2016,10,9)

MID_AUTUMN_START = date(2016,9,15)
MID_AUTUMN_END = date(2016,9,18)


TRAIN_START_DAY = date(2016,9,19)
TRAIN_END_DAY = date(2016,10,17)

VALI_START_DAY = date(2016,10,11)
VALI_END_DAY = date(2016,10,17)

TEST_START_DAY = date(2016,10,18)
TEST_END_DAY = date(2016,10,24)

In [226]:
train_df.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [227]:
def MAPE(pred, true):
    return abs((true - pred) / true)

def cal_mape(df_pred, df_true):
    pred_values = df_pred.values
    true_values = df_true.values
    mape_mean = 0.0
    for i in range(len(pred_values)):
        pred_i = pred_values[i]
        true_i = true_values[i]
        mape_mean += abs((pred_i-true_i) / true_i)
    mape_mean /= len(pred_values)
    return mape_mean

def per_20min(dt):
    minute = int(math.floor(dt.minute / 20) * 20)
    second = 0
    dt_new = datetime(dt.year, dt.month, dt.day, dt.hour,minute, 0)
    return dt_new

# 9~19～10.17只有国庆节，因此只考虑国庆节
def remove_holiday(df):
    day_all = df.time.dt.date
    df = df.loc[((day_all < NATIONNAL_START) | (day_all > NATIONNAL_END))]
    return df

def select_time(df):
    df['time'] = df.time.apply(
        per_20min)
    if {'has_etc','vehicle_type', 'vehicle_model'}.issubset(df.columns):
        df = df.drop(['has_etc','vehicle_type', 'vehicle_model'], axis=1)
    df = df.groupby(['tollgate_id', 'direction', 'time']).size()
    df = df.reset_index()
    df = df.rename_axis({0:'volume'}, axis='columns')
    hour = df.time.dt.hour
    df = df.loc[((hour >= 6) & (hour < 10)) 
                     | ((hour >= 15) & (hour < 19))]
    df = df.sort_values(['tollgate_id','direction','time'])
    return df

def slice_am_pm(df):
    hours = df.time.dt.hour
    df_am = df.loc[(hours < 12)]
    df_pm = df.loc[(hours >= 12)]
    return df_am, df_pm

def slice_time(df):
    hour = df.time.dt.hour  
    df_prev2h = df.loc[(((hour >= 6) & (hour < 8)) | ((hour >= 15) & (hour < 17)))]
    df_follow2h = df.loc[(((hour >= 8) & (hour < 10)) | ((hour >= 17) & (hour < 19)))]
    return df_prev2h, df_follow2h

def complete_miss_time(df, df_type='train'):
    start_day = df.time.dt.date.values[0]
    end_day = df.time.dt.date.values[-1]
    toll_dire = [(1,0), (1,1), (2,0), (3,0), (3,1)]
    if df_type == 'test':
        hour_min = [(6,0), (6,20), (6,40), (7,0), (7,20), (7,40),
              (15,0), (15,20), (15,40), (16,0), (16,20), (16,40)]
    else:        
        hour_min = [(6,0), (6,20), (6,40), (7,0), (7,20), (7,40),
                (8,0), (8,20), (8,40), (9,0), (9,20), (9,40),
              (15,0), (15,20), (15,40), (16,0), (16,20), (16,40),
              (17,0), (17,20), (17,40), (18,0), (18,20), (18,40)]
    df_comp = pd.DataFrame(columns=['tollgate_id','direction','time','volume'])
    for d in range((end_day - start_day).days+1):
        day = start_day + timedelta(days=d)
        if ((day < NATIONNAL_START) or (day > NATIONNAL_END)):
            for i in range(len(toll_dire)):
                toll,dire= toll_dire[i]
                for j in range(len(hour_min)):
                    h, m = hour_min[j]
                    day_time = datetime(day.year, day.month, day.day, h, m, 0)
                    index = ((df.tollgate_id == toll) & (df.direction == dire) &
                            (df.time == day_time))
                    volume = df.loc[index].volume
                    if (not volume.empty):
                        v = volume.values[0]
                    else:
                        v = np.NaN
                    row = {'tollgate_id': toll, 'direction':dire,
                       'time': str(day_time), 'volume':v} 
                    df_comp = df_comp.append(row, ignore_index=True)
    
    df_comp['tollgate_id'] = df_comp['tollgate_id'].astype(int)
    df_comp['direction'] = df_comp['direction'].astype(int)
    df_comp.time = pd.to_datetime(df_comp.time)
    df_comp['volume'] = df_comp.volume.interpolate(method='linear')
    return df_comp

def repeat_days(df, start_day, end_day):
    df_repeat = pd.DataFrame(columns=['tollgate_id','direction','time','volume'])
    for d in range((end_day - start_day).days+1):
        day = start_day + timedelta(days=d)
        temp = df.copy()
        temp.time = temp.time.apply(lambda t:
                        datetime(day.year,day.month,day.day,t.hour,t.minute,0))
        df_repeat = df_repeat.append(temp)
    
    return df_repeat

In [228]:
train_df = remove_holiday(train_df)
train_df = select_time(train_df)
train_df = complete_miss_time(train_df, df_type='train')
train_am, train_pm = slice_am_pm(train_df)

test_df = select_time(test_df)
test_df = complete_miss_time(test_df, df_type='test')
test_am, test_pm = slice_am_pm(test_df)

In [229]:
# 数据对比完毕，和官方教程一致
print(test_am.shape[0])
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())
print(len(train_df.time.dt.date.unique()))
test_am.head(20)

210
0
0
20


,tollgate_id,direction,time,volume
0,1,0,2016-10-18 06:00:00,13.0
1,1,0,2016-10-18 06:20:00,17.0
2,1,0,2016-10-18 06:40:00,21.0
3,1,0,2016-10-18 07:00:00,31.0
4,1,0,2016-10-18 07:20:00,28.0
5,1,0,2016-10-18 07:40:00,47.0
12,1,1,2016-10-18 06:00:00,37.0
13,1,1,2016-10-18 06:20:00,47.0
14,1,1,2016-10-18 06:40:00,72.0
15,1,1,2016-10-18 07:00:00,68.0


In [230]:
def generate_time_series(df_am, df_pm, df_type='train'):
    toll_dire = [(1,0), (1,1), (2,0), (3,0), (3,1)]
    df_days = df_am.time.dt.date.unique()
    df_days_len = len(df_days)
    set_am_data = {}
    set_pm_data = {}
    # 6~10(am), 15~19(pm)的12个时间段
    series_len = 12
    window_len = 7
    n_series = 6
    for i in range(len(toll_dire)):
        toll, dire = toll_dire[i]
        df_am_unit = df_am.loc[((df_am.tollgate_id == toll) & (df_am.direction == dire))]
        df_pm_unit = df_pm.loc[((df_pm.tollgate_id == toll) & (df_pm.direction == dire))]
        arr_am_data = df_am_unit.volume.values.reshape(df_days_len,-1)
        arr_pm_data = df_pm_unit.volume.values.reshape(df_days_len,-1)
        set_am_data[i] = arr_am_data
        set_pm_data[i] = arr_pm_data
    # 对于测试集或验证集，不需要用滑动窗口处理数据，直接返回原始数据
    if df_type == 'test' or df_type == 'validation':
        return set_am_data, set_pm_data
    # 对于训练集：设置滑动窗口将（1x12）数据处理成（6x7）的时间序列
    for i in range(len(set_am_data)):
        arr_am_i = set_am_data[i]
        arr_pm_i = set_pm_data[i]
        arr_am_data = []
        arr_pm_data = []
        for j in range(df_days_len):
            arr_am_i_j = arr_am_i[j,:]
            arr_pm_i_j = arr_pm_i[j,:]
            for k in range(n_series):
                arr_am_data.append(arr_am_i_j[k:(k+window_len)])
                arr_pm_data.append(arr_pm_i_j[k:(k+window_len)])
        arr_am_data = np.array(arr_am_data).reshape((df_days_len*n_series),-1)
        arr_pm_data = np.array(arr_pm_data).reshape((df_days_len*n_series),-1)
        set_am_data[i] = arr_am_data
        set_pm_data[i] = arr_pm_data
    return set_am_data, set_pm_data

In [231]:
train_set_am, train_set_pm = generate_time_series(train_am, train_pm, df_type='train')
test_set_am, test_set_pm = generate_time_series(test_am, test_pm, df_type='test')

In [232]:
train_set_am_i = train_set_am[0]
train_set_pm_i = train_set_pm[0]
print(train_set_am_i.shape)
print(train_set_pm_i.shape)
print(train_set_am_i[0:2,:])
print(train_set_pm_i[0:2,:])

(120, 7)
(120, 7)
[[  8.  13.  32.  39.  31.  43.  46.]
 [ 13.  32.  39.  31.  43.  46.  56.]]
[[ 48.  57.  43.  42.  46.  55.  36.]
 [ 57.  43.  42.  46.  55.  36.  34.]]


In [233]:
test_set_am_i = test_set_am[0]
test_set_pm_i = test_set_pm[0]
print(test_set_am_i.shape)
print(test_set_pm_i.shape)
print(test_set_am_i[0:2,:])
print(test_set_pm_i[0:2,:])

(7, 6)
(7, 6)
[[ 13.  17.  21.  31.  28.  47.]
 [ 12.  16.  17.  22.  38.  41.]]
[[ 52.  38.  35.  57.  45.  53.]
 [ 40.  34.  50.  39.  40.  36.]]


In [234]:
# define hidden size of lstm cell
LSTM_SIZE = 40
EPOCHS = 10

# 使用Pytorch的LSTMCell进行预测
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, LSTM_SIZE)
        self.lstm2 = nn.LSTMCell(LSTM_SIZE, 1)
        
    def forward(self, input_data, future=0):
        outputs = []
        h_t = Variable(torch.zeros(input_data.size(0), LSTM_SIZE).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input_data.size(0), LSTM_SIZE).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input_data.size(0), 1).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input_data.size(0), 1).double(), requires_grad=False)
        for i, input_t in enumerate(input_data.chunk(input_data.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]
        # if we should prdrict the future
        for i in range(future):
            h_t, c_t = self.lstm1(c_t2, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

def lstm_predict(train_arr, test_arr):
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    train_max = np.amax(train_arr)
    test_max = np.amax(test_arr)
    train_data = train_arr / train_max
    test_data = test_arr / test_max
    input_data = Variable(torch.from_numpy(train_data[:, :-1]), requires_grad=False)
    target_data = Variable(torch.from_numpy(train_data[:, 1:]), requires_grad=False)
    vali_data = Variable(torch.from_numpy(test_data), requires_grad=False)
    # build the model
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters())
    # begin to train
    print(vali_data[:2])
    for i in range(EPOCHS):
#         print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out_data = seq(input_data)
            loss = criterion(out_data, target_data)
#             print('\tloss:', loss.data.numpy()[0])
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict
        future = 6 
        pred = seq(vali_data, future=future)
        y = pred.data.numpy()*test_max
        # draw the result
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30) 
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(vali_data.size(1)), yi[:vali_data.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(vali_data.size(1), vali_data.size(1) + future), yi[vali_data.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
        plt.savefig('fig/task2_lstm_pred{}.jpg'.format(i))
        plt.close()
    return y

In [235]:
for i in range(len(range(1))):
    train_am_data = train_set_am[i]
    train_pm_data = train_set_pm[i]
    test_am_data = test_set_am[i]
    test_pm_data = test_set_pm[i]
    print(train_am_data[:2])
    print(train_pm_data[:2])
    print(test_am_data[:2])
    print(test_pm_data[:2])
    test_am_pred = lstm_predict(train_am_data, test_am_data)
#     test_pm_pred = lstm_predict(train_pm_data, test_pm_data)
   

[[  8.  13.  32.  39.  31.  43.  46.]
 [ 13.  32.  39.  31.  43.  46.  56.]]
[[ 48.  57.  43.  42.  46.  55.  36.]
 [ 57.  43.  42.  46.  55.  36.  34.]]
[[ 13.  17.  21.  31.  28.  47.]
 [ 12.  16.  17.  22.  38.  41.]]
[[ 52.  38.  35.  57.  45.  53.]
 [ 40.  34.  50.  39.  40.  36.]]
Variable containing:
 0.2766  0.3617  0.4468  0.6596  0.5957  1.0000
 0.2553  0.3404  0.3617  0.4681  0.8085  0.8723
[torch.DoubleTensor of size 2x6]

